In [1]:
%load_ext autoreload
%autoreload 2


import flwr as fl
from flwr.server.strategy.fedavg import FedAvg
from flwr.server import Server
from flwr.server.client_manager import SimpleClientManager
from typing import  List, Tuple
from flwr.common import Metrics

NUM_CLIENT = 1

def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    """Aggregation function for (federated) evaluation metrics, i.e. those returned by
    the client's evaluate() method."""
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"accuracy": sum(accuracies) / sum(examples)}

strategy = FedAvg(
    min_available_clients=NUM_CLIENT,
    min_fit_clients=NUM_CLIENT,
    min_evaluate_clients=NUM_CLIENT,
    evaluate_metrics_aggregation_fn=weighted_average,  # aggregates federated metrics
)


client_manager = SimpleClientManager()

server = Server(client_manager=client_manager, strategy=strategy)


c:\Users\drith\miniconda3\envs\3.11.9\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-08-07 23:47:36,341	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
fl.server.start_server(
    server_address="0.0.0.0:8000",
    config=fl.server.ServerConfig(num_rounds=10),
    server=server,
)

INFO :      Starting Flower server, config: num_rounds=10, no round_timeout
INFO :      Flower ECE: gRPC server running (10 rounds), SSL is disabled
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
